## Maestría en Analítica de datos
## Materia Big Data
## Clase 12

*   Nombre: José Luis Ariza
*   Código: 80125197


#INSTANCIAR / CONFIGURAR ELASTIC

In [1]:
# Instalar Elasticsearch y cliente Python
!pip install elasticsearch==8.11.0 -q

import os
import json
import requests
from datetime import datetime
from elasticsearch import Elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00


In [ ]:
CLOUD_URL = "https://55a79f5769ba4e0fa584f902639b9a95.us-central1.gcp.cloud.es.io:443"
API_KEY = "czl4ZTJKb0JTWHI2NmlYUzFIY286VUhfUmZidjdCQVcwSjdESFEybXl5Zw=="

# Crear cliente de ElasticSearch
es = Elasticsearch(
    CLOUD_URL,
    api_key=API_KEY,
    verify_certs=True
)
# Verificar conexión
if es.ping():
    print("Conexión exitosa a ElasticSearch Cloud")
    print(f"Cluster info: {es.info()['version']['number']}")
else:
    print("Error al conectar con ElasticSearch")

Conexión exitosa a ElasticSearch Cloud
Cluster info: 9.2.1


# Preparar/crear un index

In [ ]:
INDEX_NAME = "index_cuentos"
# Eliminar index si existe (para empezar limpio)
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' eliminado")

Index 'index_cuentos' eliminado


# crear mapping (configuración) del index

In [ ]:
mapping = {
    "mappings": {
        "properties": {
            "texto": {"type": "text", "analyzer": "spanish"},
            "autor": {"type": "keyword"},
            "fecha_creacion": {"type": "date"},
            "tipo_cuento": {"type": "keyword"}
        }
    }
}
es.indices.create(index=INDEX_NAME, body=mapping)
print(f" Index '{INDEX_NAME}' creado con mapping")

 Index 'index_cuentos' creado con mapping


# indexar (masívamente) documentos

In [ ]:
cuentos = [
    {
        "texto": "Había una vez un pequeño conejo llamado Tambor que vivía en el bosque encantado. Un día decidió explorar más allá de su madriguera y encontró un jardín mágico lleno de zanahorias gigantes.",
        "autor": "María González",
        "fecha_creacion": "2024-01-15",
        "tipo_cuento": "infantil"
    },
    {
        "texto": "La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían que por las noches se escuchaban gritos escalofriantes. Una noche, Juan decidió investigar y lo que encontró dentro le heló la sangre.",
        "autor": "Carlos Ruiz",
        "fecha_creacion": "2024-02-20",
        "tipo_cuento": "terror"
    },
    {
        "texto": "En el reino de Eldoria, la princesa Luna poseía el poder de controlar las estrellas. Cuando el dragón oscuro amenazó el reino, ella reunió a los guerreros elementales para salvar su mundo.",
        "autor": "Ana Martínez",
        "fecha_creacion": "2024-03-10",
        "tipo_cuento": "fantástico"
    },
    {
        "texto": "El alcalde prometió durante su campaña combatir la corrupción. Sin embargo, documentos filtrados revelaron su conexión con empresas fantasma. La población salió a las calles exigiendo transparencia.",
        "autor": "Roberto Silva",
        "fecha_creacion": "2024-04-05",
        "tipo_cuento": "político"
    },
    {
        "texto": "Pepito el elefante no podía volar como los pájaros, pero tenía el corazón más grande del bosque. Cuando sus amigos estuvieron en peligro, descubrió que su verdadero poder estaba en su valentía y amistad.",
        "autor": "Laura Ramírez",
        "fecha_creacion": "2024-05-12",
        "tipo_cuento": "infantil"
    },
    {
        "texto": "El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veía su reflejo, sino su peor pesadilla. La última persona que lo compró nunca volvió a ser la misma.",
        "autor": "Carlos Ruiz",
        "fecha_creacion": "2024-06-18",
        "tipo_cuento": "terror"
    },
    {
        "texto": "El mago Aldrin guardaba en su torre el libro de los hechizos perdidos. Cuando el portal dimensional se abrió, criaturas de otros mundos invadieron la tierra y solo él podía cerrar la brecha.",
        "autor": "Ana Martínez",
        "fecha_creacion": "2024-07-22",
        "tipo_cuento": "fantástico"
    },
    {
        "texto": "La reforma tributaria dividió al congreso. Mientras unos defendían beneficios para grandes empresas, otros exigían mayor equidad. El debate reveló las profundas fracturas del sistema político.",
        "autor": "Roberto Silva",
        "fecha_creacion": "2024-08-30",
        "tipo_cuento": "político"
    }
]

In [ ]:
# Cargar documentos en el index
for i, cuento in enumerate(cuentos):
    response = es.index(index=INDEX_NAME, id=i+1, document=cuento)
    print(f" Cuento {i+1} indexado: {response['result']}")

 Cuento 1 indexado: created
 Cuento 2 indexado: created
 Cuento 3 indexado: created
 Cuento 4 indexado: created
 Cuento 5 indexado: created
 Cuento 6 indexado: created
 Cuento 7 indexado: created
 Cuento 8 indexado: created


# consultas

In [ ]:
es.indices.refresh(index=INDEX_NAME)
print(f"\n {len(cuentos)} cuentos cargados exitosamente")


 8 cuentos cargados exitosamente


## consultar todos los ducumentos

In [ ]:
query1 = {
    "query": {
        "match_all": {}
    }
}
result = es.search(index=INDEX_NAME, body=query1)
print(f"Total de documentos: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['autor']}: {hit['_source']['tipo_cuento']}")

Total de documentos: 8
- Laura Ramírez: infantil
- Ana Martínez: fantástico
- Carlos Ruiz: terror
- Ana Martínez: fantástico
- Roberto Silva: político
- María González: infantil
- Carlos Ruiz: terror
- Roberto Silva: político


## búsqueda por una columna (caracteristica) que tiene todos los documentos

In [ ]:
query2 = {
    "query": {
        "term": {
            "tipo_cuento": "terror"
        }
    }
}
result = es.search(index=INDEX_NAME, body=query2)
print(f"Cuentos de terror encontrados: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- '{hit['_source']['texto'][:80]}...'")
    print(f"  Autor: {hit['_source']['autor']}\n")

Cuentos de terror encontrados: 2
- 'La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían qu...'
  Autor: Carlos Ruiz

- 'El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veí...'
  Autor: Carlos Ruiz



In [ ]:
query2 = {
    "query": {
        "term": {
            "autor": "Carlos Ruiz"
        }
    }
}
result = es.search(index=INDEX_NAME, body=query2)
print(f"Cuentos de Carlos Ruiz encontrados: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- '{hit['_source']['texto'][:80]}...'")
    print(f"  Autor: {hit['_source']['tipo_cuento']}\n")

Cuentos de Carlos Ruiz encontrados: 2
- 'La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían qu...'
  Autor: terror

- 'El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veí...'
  Autor: terror



# Búsquedas dinámicas (match)

In [ ]:
query3 = {
    "query": {
        "match": {
            "texto": "dragón bosque mágico"
        }
    }
}
result = es.search(index=INDEX_NAME, body=query3)
print(f"Documentos que contienen 'dragón', 'bosque' o 'mágico': {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- Score: {hit['_score']:.2f} | Tipo: {hit['_source']['tipo_cuento']}")
    print(f"  Texto: {hit['_source']['texto'][:100]}...\n")
    print(f"  Texto: {hit['_source']['autor'][:100]}...\n")

Documentos que contienen 'dragón', 'bosque' o 'mágico': 0


## búsquedas con rango de fechas.

In [ ]:
query5 = {
    "query": {
        "range": {
            "fecha_creacion": {
                "gte": "2024-04-01",
                "lte": "2024-07-31"
            }
        }
    }
}
result = es.search(index=INDEX_NAME, body=query5)
print(f"Cuentos entre abril y julio 2024: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['fecha_creacion']} | {hit['_source']['tipo_cuento']} | {hit['_source']['autor']}")

Cuentos entre abril y julio 2024: 4
- 2024-04-05 | político | Roberto Silva
- 2024-05-12 | infantil | Laura Ramírez
- 2024-06-18 | terror | Carlos Ruiz
- 2024-07-22 | fantástico | Ana Martínez


## búsqueda booleana compuesta

In [ ]:
query6 = {
    "query": {
        "bool": {
            "must": [
                {"match": {"texto": "reino"}}
            ],
            "filter": [
                {"term": {"tipo_cuento": "fantástico"}}
            ]
        }
    }
}
result = es.search(index=INDEX_NAME, body=query6)
print(f"Cuentos fantásticos que mencionan 'reino': {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['autor']}: {hit['_source']['texto'][:100]}...")

Cuentos fantásticos que mencionan 'reino': 1
- Ana Martínez: En el reino de Eldoria, la princesa Luna poseía el poder de controlar las estrellas. Cuando el dragó...


# consultas con agregaciones (filtros)

In [ ]:
agg1 = {
    "size": 0,
    "aggs": {
        "cuentos_por_tipo": {
            "terms": {
                "field": "tipo_cuento",
                "size": 10
            }
        }
    }
}
result = es.search(index=INDEX_NAME, body=agg1)
for bucket in result['aggregations']['cuentos_por_tipo']['buckets']:
    print(f"- {bucket['key']}: {bucket['doc_count']} cuentos")

- fantástico: 2 cuentos
- infantil: 2 cuentos
- político: 2 cuentos
- terror: 2 cuentos


In [ ]:
agg2 = {
    "size": 0,
    "aggs": {
        "cuentos_por_autor": {
            "terms": {
                "field": "autor",
                "size": 10
            }
        }
    }
}
result = es.search(index=INDEX_NAME, body=agg2)
for bucket in result['aggregations']['cuentos_por_autor']['buckets']:
    print(f"- {bucket['key']}: {bucket['doc_count']} cuentos")

- Ana Martínez: 2 cuentos
- Carlos Ruiz: 2 cuentos
- Roberto Silva: 2 cuentos
- Laura Ramírez: 1 cuentos
- María González: 1 cuentos


# filtros por fecha (por mes)

In [ ]:
agg3 = {
    "size": 0,
    "aggs": {
        "cuentos_por_mes": {
            "date_histogram": {
                "field": "fecha_creacion",
                "calendar_interval": "month"
            }
        }
    }
}
result = es.search(index=INDEX_NAME, body=agg3)
for bucket in result['aggregations']['cuentos_por_mes']['buckets']:
    fecha = bucket['key_as_string']
    print(f"- {fecha}: {bucket['doc_count']} cuentos")

- 2024-01-01T00:00:00.000Z: 1 cuentos
- 2024-02-01T00:00:00.000Z: 1 cuentos
- 2024-03-01T00:00:00.000Z: 1 cuentos
- 2024-04-01T00:00:00.000Z: 1 cuentos
- 2024-05-01T00:00:00.000Z: 1 cuentos
- 2024-06-01T00:00:00.000Z: 1 cuentos
- 2024-07-01T00:00:00.000Z: 1 cuentos
- 2024-08-01T00:00:00.000Z: 1 cuentos
